## Data Augmentation

In [2]:
import os
import random
from PIL import Image, ImageEnhance, ImageOps
import numpy as np

In [3]:
# Define augmentation functions
def random_rotation(image):
    angle = random.uniform(-20, 20)  # Rotate between -30 to 30 degrees
    return image.rotate(angle)

def random_flip(image):
    if random.choice([True, False]):
        return ImageOps.mirror(image)
    return image

def random_brightness(image):
    enhancer = ImageEnhance.Brightness(image)
    factor = random.uniform(0.7, 1.3)  # Brightness factor
    return enhancer.enhance(factor)

def random_contrast(image):
    enhancer = ImageEnhance.Contrast(image)
    factor = random.uniform(0.7, 1.3)  # Contrast factor
    return enhancer.enhance(factor)

def add_random_noise(image):
    np_image = np.array(image)
    noise = np.random.normal(0, 25, np_image.shape).astype(np.int16)
    noisy_image = np.clip(np_image + noise, 0, 255).astype(np.uint8)
    return Image.fromarray(noisy_image)

def augment_image(image):
    image = random_rotation(image)
    image = random_flip(image)
    # image = random_brightness(image)
    # image = random_contrast(image)
    # image = add_random_noise(image)
    return image


In [4]:
input_folder = "C:/Users/User/OneDrive/Documents/SignLanguageApp/SLangDataset/new_Blmark_data"
output_folder = "C:/Users/User/OneDrive/Documents/SignLanguageApp/SLangDataset/new_Blmark_data_augmented"
os.makedirs(output_folder, exist_ok=True)

In [5]:
# Iterate over all subfolders and images
for subdir, _, files in os.walk(input_folder):
    relative_path = os.path.relpath(subdir, input_folder)
    output_subdir = os.path.join(output_folder, relative_path)
    os.makedirs(output_subdir, exist_ok=True)

    for file in files:
        if file.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'tiff')):
            input_path = os.path.join(subdir, file)
            output_path = os.path.join(output_subdir, file)

            try:
                with Image.open(input_path) as img:
                    img = img.convert("L")  # Ensure greyscale (black and white)
                    augmented_img = augment_image(img)
                    augmented_img.save(output_path)
            except Exception as e:
                print(f"Error processing {input_path}: {e}")

print("Data augmentation completed!")


Data augmentation completed!


## Joining Augmented Data with Original Data

In [6]:
import os
import shutil

dataset1 = r"C:\Users\User\OneDrive\Documents\SignLanguageApp\SLangDataset\new_Blmark_data"
dataset2 = r"C:\Users\User\OneDrive\Documents\SignLanguageApp\SLangDataset\new_Blmark_data_augmented"
output_dataset = r"C:\Users\User\OneDrive\Documents\SignLanguageApp\SLangDataset\new_Blmark_data_combined"

# Create the output directory if it doesn't exist
os.makedirs(output_dataset, exist_ok=True)

# Function to merge datasets with renaming
def merge_datasets(source_dir, target_dir, suffix=""):
    for class_name in os.listdir(source_dir):
        source_class_path = os.path.join(source_dir, class_name)
        target_class_path = os.path.join(target_dir, class_name)
        
        if os.path.isdir(source_class_path):
            # Create the class folder in the target if it doesn't exist
            if not os.path.exists(target_class_path):
                os.makedirs(target_class_path)
            
            for file_name in os.listdir(source_class_path):
                source_file_path = os.path.join(source_class_path, file_name)
                # Add the specified suffix to the file name
                base_name, ext = os.path.splitext(file_name)
                file_name = f"{base_name}{suffix}{ext}"
                target_file_path = os.path.join(target_class_path, file_name)
                
                # Copy the file to the target directory
                shutil.copy2(source_file_path, target_file_path)

# Merge the main dataset
merge_datasets(dataset1, output_dataset)

# Merge the augmented dataset with "_AUG" renaming
merge_datasets(dataset2, output_dataset, suffix="_AUG")


print(f"Datasets merged into: {output_dataset}")

Datasets merged into: C:\Users\User\OneDrive\Documents\SignLanguageApp\SLangDataset\new_Blmark_data_combined


### Run Trainer3.py for training the new data. Run to open UI.

In [1]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available: 1
